In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import pandas as pd
from fastcore.xtras import load_pickle

from gptchem.evaluator import evaluate_photoswitch_smiles_pred, evaluate_generated_smiles

from glob import glob

In [3]:
all_res = glob('out/**/*.pkl')

In [6]:
res = load_pickle(all_res[0])

In [17]:
res['res_at_temp'][0].keys()

dict_keys(['completions', 'generated_smiles', 'train_smiles', 'frac_valid', 'frac_unique', 'frac_smiles_in_train', 'frac_smiles_in_pubchem', 'in_pubchem', 'kld', 'frechet_d', 'frechet_score', 'valid_smiles', 'valid_indices', 'e_pi_pi_star_metrics', 'z_pi_pi_star_metrics'])

In [16]:
evaluate_generated_smiles(res['res_at_temp'][0]['generated_smiles'], res['res_at_temp'][0]['train_smiles'])

The data appears to lie in a lower-dimensional subspace of the space in which it is expressed. This has resulted in a singular data covariance matrix, which cannot be treated using the algorithms implemented in `gaussian_kde`. Consider performing principle component analysis / dimensionality reduction and using `gaussian_kde` with the transformed data.


{'frac_valid': 1.0,
 'frac_unique': 0.021739130434782608,
 'frac_smiles_in_train': 0.0,
 'frac_smiles_in_pubchem': 1.0,
 'in_pubchem': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91],
 'kld': nan,
 'frechet_d': 35.68358542233931,
 'frechet_score': 0.0007953589112722479,
 'valid_smiles': ['CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
  'CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
  'CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
  'CC1=C(C(C)=NN1)/N=N/C2=CC=C(C(F)(F)F)C=C2',
  'CC1=C(C(C)=NN1)/N=N/C2=CC=C(C